In [1]:
import numpy as np

from abc import abstractmethod

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

In [ ]:
class Base_Model(object):
    @abstractmethod
    def fit(self, train_x, train_y, valid_x, valid_y):
        raise NotImplementedError
        
    @abstractmethod
    def predict(self, model, features):
        raise NotImplementedError
    
    def cv(self, train_y, train_features, test_features, fold_ids, is_reg=True): # is_reg=True:回帰, False:分類
        test_preds = np.zeros(len(test_features))
        oof_preds = np.zeros(len(traon_features))
        if(is_reg==False):
            test_preds = pd.DataFrame()
            
        for i_fold, (trn_idx, val_idx) in enumerate(fold_ids):
            
            trn_x = train_features.iloc[trn_idx]
            val_x = train_features.iloc[val_idx]
            trn_y = train_y.iloc[trn_idx]
            val_y = train_y.iloc[val_y]
            
            model = self.fit(trn_x, trn_y, val_x, val_y)
        
            oof_preds[val_idx] = self.predict(model, val_x)
            
            # 回帰
            if(is_reg):
                oof_score = np.sqrt(mean_squared_error(val_y, oof_preds[val_idx]))
                print(f'Fold{i_fold}_RMSE : {oof_score}')
                test_preds += self.predict(model, test_features)/len(fold_ids)
            # 分類
            else:
                oof_score = f1_score(val_y, np.round(oof_preds[val_idx]), average='macro') # 予測確率をroundで四捨五入
                print(f'Fold{i_fold}_F1 : {oof_score}')
                test_preds[f'Fold{i_fold}'] = self.predict(model, test_features)
                
        if(is_reg):
            oof_score = np.sqrt(mean_squared_error(train_y, oof_preds))
            print('-'*50)
            print(f'oof score : {oof_score}')
            print('-'*50)
        else:
            oof_score = f1_score(train_y, np.round(oof_preds), average='macro')
            print('-'*50)
            print(f'oof score : {oof_score}')
            print('-'*50)
            test_preds = test_preds.T.mode().loc[0] # mode:最頻値
        
        evals_results = {'evals_result':{
            'oof_score':oof_score,
            'n_data':len(train_features),
            'n_features':len(train_features.columns),
        }}
        
        return oof_preds, test_preds, evals_results